In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy import sparse 
import time


from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import os
from sklearn.feature_selection import chi2, SelectPercentile
import lightgbm as lgb

In [2]:
train = pd.read_pickle(FILE.train_final.value)
test = pd.read_pickle(FILE.test_final.value)
train.drop('time_hour',axis = 1,inplace=True)
test.drop('time_hour',axis = 1,inplace=True)
data = pd.concat([train, test], axis=0, ignore_index=True,sort=False)


In [3]:
data = data.fillna(-1)
os.environ['TZ'] = 'Asia/Shanghai'
data['day'] = data['time'].apply(lambda x: int(time.strftime("%d", time.localtime(x))))
data['hour'] = data['time'].apply(lambda x: int(time.strftime("%H", time.localtime(x))))
data['label'] = data.click.astype(int)
del data['click']

In [4]:
bool_feature = ['creative_is_jump', 'creative_is_download', 'creative_is_js', 'creative_is_voicead',
                'creative_has_deeplink', 'app_paid']
for i in bool_feature:
    data[i] = data[i].astype(int)

data['advert_industry_inner_1'] = data['advert_industry_inner'].apply(lambda x: x.split('_')[0])

ad_cate_feature = ['adid', 'advert_id', 'orderid', 'advert_industry_inner_1', 'advert_industry_inner', 'advert_name',
                   'campaign_id', 'creative_id', 'creative_type', 'creative_tp_dnf', 'creative_has_deeplink',
                   'creative_is_jump', 'creative_is_download']

media_cate_feature = ['app_cate_id', 'f_channel', 'app_id', 'inner_slot_id']

content_cate_feature = ['city', 'carrier', 'province', 'nnt', 'devtype', 'osv', 'os', 'make', 'model']

origin_cate_list = ad_cate_feature + media_cate_feature + content_cate_feature

for i in origin_cate_list:
    data[i] = data[i].map(dict(zip(data[i].unique(), range(0, data[i].nunique()))))
    
cate_feature = origin_cate_list

num_feature = ['creative_width', 'creative_height', 'hour']

feature = cate_feature + num_feature
print(len(feature), feature)

29 ['adid', 'advert_id', 'orderid', 'advert_industry_inner_1', 'advert_industry_inner', 'advert_name', 'campaign_id', 'creative_id', 'creative_type', 'creative_tp_dnf', 'creative_has_deeplink', 'creative_is_jump', 'creative_is_download', 'app_cate_id', 'f_channel', 'app_id', 'inner_slot_id', 'city', 'carrier', 'province', 'nnt', 'devtype', 'osv', 'os', 'make', 'model', 'creative_width', 'creative_height', 'hour']


In [5]:
predict = data[data.label == -1].copy()
predict_result = predict[['instance_id']].copy()
predict_result['predicted_score'] = 0
predict_x = predict.drop('label', axis=1)

train_x = data[data.label != -1].copy()
train_y = data[data.label != -1].label.values

# prepare one-hot

In [6]:
special = ['model','make']
try:
    raise FileNotFoundError
    base_train_csr = pickle.load(open(FILE.one_hot_base_train.value,'rb'))
    base_predict_csr = pickle.load(open(FILE.one_hot_base_test.value,'rb'))
    print('historical base was loaded!')
except Exception:
    print('not found one_hot from history! generate from scratch!')
    base_train_csr = sparse.csr_matrix((len(train), 0))
    base_predict_csr = sparse.csr_matrix((len(test), 0))
    
    enc = OneHotEncoder()
    for feature in tqdm(cate_feature):
        try:
            onehot_train = pickle.load(open(FILE.one_hot_train_formatter.value.format(feature),'rb'))
            onehot_test = pickle.load(open(FILE.one_hot_test_formatter.value.format(feature),'rb'))
            if feature in special:
                pass
#                 raise FileNotFoundError
        except FileNotFoundError:
            print(feature)
                
            enc.fit(data[feature].values.reshape(-1, 1))
            onehot_train = enc.transform(train_x[feature].values.reshape(-1, 1))
            print(onehot_train.shape)
            onehot_test = enc.transform(predict_x[feature].values.reshape(-1, 1))
            pickle.dump(onehot_train,open(FILE.one_hot_train_formatter.value.format(feature),'wb'))
            pickle.dump(onehot_test,open(FILE.one_hot_test_formatter.value.format(feature),'wb'))


        base_train_csr = sparse.hstack((base_train_csr, onehot_train), 'csr',
                                       'bool')
        base_predict_csr = sparse.hstack((base_predict_csr, onehot_test),
                                         'csr',
                                         'bool')
    print('one-hot prepared !')

    cv = CountVectorizer(min_df=20)
    
    for feature in ['user_tags']:
        try:
            onehot_train = pickle.load(open(FILE.one_hot_train_formatter.value.format(feature),'rb'))
            onehot_test = pickle.load(open(FILE.one_hot_test_formatter.value.format(feature),'rb'))
#             raise FileNotFoundError
        except FileNotFoundError:
            
            data[feature] = data[feature].astype(str)
            cv.fit(data[feature])
            onehot_train = cv.transform(train_x[feature].astype(str))
            onehot_test = cv.transform(predict_x[feature].astype(str))
            pickle.dump(onehot_train,open(FILE.one_hot_train_formatter.value.format(feature),'wb'))
            pickle.dump(onehot_test,open(FILE.one_hot_test_formatter.value.format(feature),'wb'))
    print(base_train_csr.shape)
    base_train_csr = sparse.hstack((base_train_csr, onehot_train), 'csr', 'bool')
    base_predict_csr = sparse.hstack((base_predict_csr, onehot_test), 'csr',
                             'bool')
    pickle.dump(base_train_csr,open(FILE.one_hot_base_train.value,'wb'))
    pickle.dump(base_predict_csr,open(FILE.one_hot_base_test.value,'wb'))

    
print(base_train_csr.shape)
print(base_predict_csr.shape)


  4%|▍         | 1/26 [00:00<00:02,  9.14it/s]

not found one_hot from history! generate from scratch!


100%|██████████| 26/26 [00:22<00:00,  1.15it/s]


one-hot prepared !
(2992639, 32549)
(2992639, 33966)
(80276, 33966)


In [7]:
start = 32549
end = 33966
tail = end - start

In [9]:
train_csr = sparse.hstack(
    (sparse.csr_matrix(train_x[num_feature]), base_train_csr), 'csr').astype(
    'float32')
predict_csr = sparse.hstack(
    (sparse.csr_matrix(predict_x[num_feature]), base_predict_csr), 'csr').astype('float32')
print(train_csr.shape)
# feature_select = SelectPercentile(chi2, percentile=99)
# feature_select.fit(train_csr, train_y)
# train_csr = feature_select.transform(train_csr)
# predict_csr = feature_select.transform(predict_csr)
# print('feature select')
# print(train_csr.shape)

(2992639, 33969)


# Define LGB

In [11]:
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=32, reg_alpha=0, reg_lambda=0.1,
    max_depth=-1, n_estimators=5000, objective='binary',
    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.05, random_state=2018, n_jobs=16
)


In [12]:
skf = StratifiedKFold(n_splits=5, random_state=1001, shuffle=True)
best_score = []
feature_importance_tmp_split = []
feature_importance_tmp_gain = []
df_fea = pd.DataFrame()
for index, (train_index, test_index) in enumerate(skf.split(train_csr, train_y)):
    lgb_model.fit(train_csr[train_index], train_y[train_index],
                  eval_set=[(train_csr[train_index], train_y[train_index]),
                            (train_csr[test_index], train_y[test_index])], early_stopping_rounds=100,verbose=10)
    best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
    print(best_score)
    test_pred = lgb_model.predict_proba(predict_csr, num_iteration=lgb_model.best_iteration_)[:, 1]
    print('test mean:', test_pred.mean())
    predict_result['predicted_score'] = predict_result['predicted_score'] + test_pred
    feature_importance_tmp_split.append(lgb_model.booster_.feature_importance(importance_type='split'))
    feature_importance_tmp_gain.append(lgb_model.booster_.feature_importance(importance_type='gain'))
    
df_fea['col_name'] = lgb_model.booster_.feature_name()
df_fea['split'] = np.mean(feature_importance_tmp_split,axis=0)
df_fea['gain'] = np.mean(feature_importance_tmp_gain,axis=0)

Training until validation scores don't improve for 100 rounds.
[10]	valid_0's binary_logloss: 0.458342	valid_1's binary_logloss: 0.458393
[20]	valid_0's binary_logloss: 0.440414	valid_1's binary_logloss: 0.440549
[30]	valid_0's binary_logloss: 0.431592	valid_1's binary_logloss: 0.43179
[40]	valid_0's binary_logloss: 0.426908	valid_1's binary_logloss: 0.427147
[50]	valid_0's binary_logloss: 0.424297	valid_1's binary_logloss: 0.424566
[60]	valid_0's binary_logloss: 0.422663	valid_1's binary_logloss: 0.422967
[70]	valid_0's binary_logloss: 0.421628	valid_1's binary_logloss: 0.421959
[80]	valid_0's binary_logloss: 0.420872	valid_1's binary_logloss: 0.421219
[90]	valid_0's binary_logloss: 0.420245	valid_1's binary_logloss: 0.420626
[100]	valid_0's binary_logloss: 0.419763	valid_1's binary_logloss: 0.42017
[110]	valid_0's binary_logloss: 0.41935	valid_1's binary_logloss: 0.419794
[120]	valid_0's binary_logloss: 0.419004	valid_1's binary_logloss: 0.419464
[130]	valid_0's binary_logloss: 0.418

[1550]	valid_0's binary_logloss: 0.408782	valid_1's binary_logloss: 0.41522
[1560]	valid_0's binary_logloss: 0.40874	valid_1's binary_logloss: 0.415218
[1570]	valid_0's binary_logloss: 0.408695	valid_1's binary_logloss: 0.415213
[1580]	valid_0's binary_logloss: 0.408652	valid_1's binary_logloss: 0.415213
[1590]	valid_0's binary_logloss: 0.40861	valid_1's binary_logloss: 0.415209
[1600]	valid_0's binary_logloss: 0.408568	valid_1's binary_logloss: 0.415208
[1610]	valid_0's binary_logloss: 0.408528	valid_1's binary_logloss: 0.415204
[1620]	valid_0's binary_logloss: 0.408488	valid_1's binary_logloss: 0.415202
[1630]	valid_0's binary_logloss: 0.408442	valid_1's binary_logloss: 0.415203
[1640]	valid_0's binary_logloss: 0.408393	valid_1's binary_logloss: 0.415196
[1650]	valid_0's binary_logloss: 0.408351	valid_1's binary_logloss: 0.415197
[1660]	valid_0's binary_logloss: 0.408307	valid_1's binary_logloss: 0.415192
[1670]	valid_0's binary_logloss: 0.408265	valid_1's binary_logloss: 0.415191
[1

[200]	valid_0's binary_logloss: 0.417285	valid_1's binary_logloss: 0.417481
[210]	valid_0's binary_logloss: 0.417097	valid_1's binary_logloss: 0.417321
[220]	valid_0's binary_logloss: 0.416942	valid_1's binary_logloss: 0.417209
[230]	valid_0's binary_logloss: 0.416789	valid_1's binary_logloss: 0.41709
[240]	valid_0's binary_logloss: 0.416653	valid_1's binary_logloss: 0.416991
[250]	valid_0's binary_logloss: 0.416526	valid_1's binary_logloss: 0.416895
[260]	valid_0's binary_logloss: 0.416397	valid_1's binary_logloss: 0.416804
[270]	valid_0's binary_logloss: 0.416275	valid_1's binary_logloss: 0.416714
[280]	valid_0's binary_logloss: 0.416168	valid_1's binary_logloss: 0.416642
[290]	valid_0's binary_logloss: 0.416063	valid_1's binary_logloss: 0.416574
[300]	valid_0's binary_logloss: 0.415961	valid_1's binary_logloss: 0.416515
[310]	valid_0's binary_logloss: 0.415854	valid_1's binary_logloss: 0.416442
[320]	valid_0's binary_logloss: 0.415754	valid_1's binary_logloss: 0.416387
[330]	valid_0

[1280]	valid_0's binary_logloss: 0.410072	valid_1's binary_logloss: 0.414918
[1290]	valid_0's binary_logloss: 0.410022	valid_1's binary_logloss: 0.414917
[1300]	valid_0's binary_logloss: 0.409976	valid_1's binary_logloss: 0.414914
[1310]	valid_0's binary_logloss: 0.409929	valid_1's binary_logloss: 0.414913
[1320]	valid_0's binary_logloss: 0.40988	valid_1's binary_logloss: 0.414909
[1330]	valid_0's binary_logloss: 0.409831	valid_1's binary_logloss: 0.414902
[1340]	valid_0's binary_logloss: 0.409785	valid_1's binary_logloss: 0.414904
[1350]	valid_0's binary_logloss: 0.409739	valid_1's binary_logloss: 0.414896
[1360]	valid_0's binary_logloss: 0.409692	valid_1's binary_logloss: 0.414889
[1370]	valid_0's binary_logloss: 0.409648	valid_1's binary_logloss: 0.414886
[1380]	valid_0's binary_logloss: 0.409602	valid_1's binary_logloss: 0.414885
[1390]	valid_0's binary_logloss: 0.409556	valid_1's binary_logloss: 0.414883
[1400]	valid_0's binary_logloss: 0.409512	valid_1's binary_logloss: 0.414876


[2350]	valid_0's binary_logloss: 0.40556	valid_1's binary_logloss: 0.414714
[2360]	valid_0's binary_logloss: 0.405519	valid_1's binary_logloss: 0.414717
[2370]	valid_0's binary_logloss: 0.405483	valid_1's binary_logloss: 0.414718
[2380]	valid_0's binary_logloss: 0.405443	valid_1's binary_logloss: 0.414717
[2390]	valid_0's binary_logloss: 0.405405	valid_1's binary_logloss: 0.414717
[2400]	valid_0's binary_logloss: 0.405367	valid_1's binary_logloss: 0.414717
[2410]	valid_0's binary_logloss: 0.405327	valid_1's binary_logloss: 0.414718
[2420]	valid_0's binary_logloss: 0.405292	valid_1's binary_logloss: 0.414715
[2430]	valid_0's binary_logloss: 0.405256	valid_1's binary_logloss: 0.414715
Early stopping, best iteration is:
[2330]	valid_0's binary_logloss: 0.405634	valid_1's binary_logloss: 0.414712
[0.4151058938784851, 0.4147120174092003]
test mean: 0.21322968381185908
Training until validation scores don't improve for 100 rounds.
[10]	valid_0's binary_logloss: 0.458339	valid_1's binary_logl

...

[680]	valid_0's binary_logloss: 0.413178	valid_1's binary_logloss: 0.415632
[690]	valid_0's binary_logloss: 0.413118	valid_1's binary_logloss: 0.415622
[700]	valid_0's binary_logloss: 0.41306	valid_1's binary_logloss: 0.415609
[710]	valid_0's binary_logloss: 0.413005	valid_1's binary_logloss: 0.4156
[720]	valid_0's binary_logloss: 0.412941	valid_1's binary_logloss: 0.415579
[730]	valid_0's binary_logloss: 0.412885	valid_1's binary_logloss: 0.415565
[740]	valid_0's binary_logloss: 0.412833	valid_1's binary_logloss: 0.415555
[750]	valid_0's binary_logloss: 0.41278	valid_1's binary_logloss: 0.415544
[760]	valid_0's binary_logloss: 0.412725	valid_1's binary_logloss: 0.415542
[770]	valid_0's binary_logloss: 0.412667	valid_1's binary_logloss: 0.415525
[780]	valid_0's binary_logloss: 0.412612	valid_1's binary_logloss: 0.415516
[790]	valid_0's binary_logloss: 0.412559	valid_1's binary_logloss: 0.415509
[800]	valid_0's binary_logloss: 0.412498	valid_1's binary_logloss: 0.415499
[810]	valid_0's 

[1760]	valid_0's binary_logloss: 0.40793	valid_1's binary_logloss: 0.415001
[1770]	valid_0's binary_logloss: 0.40789	valid_1's binary_logloss: 0.414998
[1780]	valid_0's binary_logloss: 0.407849	valid_1's binary_logloss: 0.414997
[1790]	valid_0's binary_logloss: 0.407805	valid_1's binary_logloss: 0.414993
[1800]	valid_0's binary_logloss: 0.40776	valid_1's binary_logloss: 0.414985
[1810]	valid_0's binary_logloss: 0.407715	valid_1's binary_logloss: 0.414984
[1820]	valid_0's binary_logloss: 0.407674	valid_1's binary_logloss: 0.41498
[1830]	valid_0's binary_logloss: 0.40763	valid_1's binary_logloss: 0.414978
[1840]	valid_0's binary_logloss: 0.407589	valid_1's binary_logloss: 0.414973
[1850]	valid_0's binary_logloss: 0.40755	valid_1's binary_logloss: 0.41497
[1860]	valid_0's binary_logloss: 0.407506	valid_1's binary_logloss: 0.414968
[1870]	valid_0's binary_logloss: 0.407464	valid_1's binary_logloss: 0.414968
[1880]	valid_0's binary_logloss: 0.407421	valid_1's binary_logloss: 0.414968
[1890]

[110]	valid_0's binary_logloss: 0.419346	valid_1's binary_logloss: 0.419708
[120]	valid_0's binary_logloss: 0.419009	valid_1's binary_logloss: 0.4194
[130]	valid_0's binary_logloss: 0.418701	valid_1's binary_logloss: 0.419122
[140]	valid_0's binary_logloss: 0.41844	valid_1's binary_logloss: 0.418888
[150]	valid_0's binary_logloss: 0.418192	valid_1's binary_logloss: 0.418675
[160]	valid_0's binary_logloss: 0.417943	valid_1's binary_logloss: 0.418453
[170]	valid_0's binary_logloss: 0.417734	valid_1's binary_logloss: 0.41827
[180]	valid_0's binary_logloss: 0.41754	valid_1's binary_logloss: 0.418105
[190]	valid_0's binary_logloss: 0.417362	valid_1's binary_logloss: 0.417955
[200]	valid_0's binary_logloss: 0.4172	valid_1's binary_logloss: 0.417833
[210]	valid_0's binary_logloss: 0.417044	valid_1's binary_logloss: 0.417709
[220]	valid_0's binary_logloss: 0.416885	valid_1's binary_logloss: 0.417592
[230]	valid_0's binary_logloss: 0.416738	valid_1's binary_logloss: 0.417475
[240]	valid_0's bin

[1200]	valid_0's binary_logloss: 0.410383	valid_1's binary_logloss: 0.415356
[1210]	valid_0's binary_logloss: 0.41034	valid_1's binary_logloss: 0.415353
[1220]	valid_0's binary_logloss: 0.410293	valid_1's binary_logloss: 0.415349
[1230]	valid_0's binary_logloss: 0.410247	valid_1's binary_logloss: 0.415349
[1240]	valid_0's binary_logloss: 0.410202	valid_1's binary_logloss: 0.415343
[1250]	valid_0's binary_logloss: 0.410152	valid_1's binary_logloss: 0.415337
[1260]	valid_0's binary_logloss: 0.410102	valid_1's binary_logloss: 0.415337
[1270]	valid_0's binary_logloss: 0.41005	valid_1's binary_logloss: 0.415327
[1280]	valid_0's binary_logloss: 0.409999	valid_1's binary_logloss: 0.415326
[1290]	valid_0's binary_logloss: 0.409948	valid_1's binary_logloss: 0.415321
[1300]	valid_0's binary_logloss: 0.409903	valid_1's binary_logloss: 0.415318
[1310]	valid_0's binary_logloss: 0.409856	valid_1's binary_logloss: 0.415314
[1320]	valid_0's binary_logloss: 0.409808	valid_1's binary_logloss: 0.415312
[

test mean: 0.21334081971780192


# Experiment

In [17]:
utag_20_index = list(df_fea.iloc[-tail:].sort_values('split',ascending=False).iloc[:20].index)

In [18]:
train_tag = train_csr[:,utag_20_index]
test_tag = predict_csr[:,utag_20_index]

In [19]:
from scipy.sparse import csr_matrix
def save_csr(matrix,file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    np.save(data_file,matrix.data)
    print('data save')
    np.save(indices_file,matrix.indices)
    print('indices save')
    np.save(indptr_file,matrix.indptr)
    print('indptr save')
    print('save done!')
    
save_csr(train_tag,'../../data/features/one_hot/train_utag_top20')
save_csr(test_tag,'../../data/features/one_hot/test_utag_top20')

data save
indices save
indptr save
save done!
data save
indices save
indptr save
save done!


In [20]:
# https://10.0.24.37:8888/tree/weiluo/AlphaBoosting/xun_fei_ads/luowei/data/features/one_hot
# train_utag_20 = load_csr('/homw/kai/data/weiluo/AlphaBoosting/xun_fei_ads/luowei/data/features/one_hot/train_utag_top20')
# test_utag_20 = load_csr('/homw/kai/data/weiluo/AlphaBoosting/xun_fei_ads/luowei/data/features/one_hot/train_utag_top20')

In [36]:
skf = StratifiedKFold(n_splits=5, random_state=1001, shuffle=True)
best_score = []
for index, (train_index, test_index) in enumerate(skf.split(train_csr, train_y)):
    lgb_model.fit(train_csr[train_index], train_y[train_index],
                  eval_set=[(train_csr[train_index], train_y[train_index]),
                            (train_csr[test_index], train_y[test_index])], early_stopping_rounds=100,verbose=10)
    best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
    print(best_score)
    test_pred = lgb_model.predict_proba(predict_csr, num_iteration=lgb_model.best_iteration_)[:, 1]
    print('test mean:', test_pred.mean())
    predict_result['predicted_score'] = predict_result['predicted_score'] + test_pred

Training until validation scores don't improve for 100 rounds.
[10]	valid_0's binary_logloss: 0.458444	valid_1's binary_logloss: 0.458489
[20]	valid_0's binary_logloss: 0.440393	valid_1's binary_logloss: 0.440515
[30]	valid_0's binary_logloss: 0.431551	valid_1's binary_logloss: 0.431738
[40]	valid_0's binary_logloss: 0.426854	valid_1's binary_logloss: 0.427084
[50]	valid_0's binary_logloss: 0.424259	valid_1's binary_logloss: 0.424508
[60]	valid_0's binary_logloss: 0.422659	valid_1's binary_logloss: 0.422944
[70]	valid_0's binary_logloss: 0.421626	valid_1's binary_logloss: 0.421931
[80]	valid_0's binary_logloss: 0.420855	valid_1's binary_logloss: 0.421194
[90]	valid_0's binary_logloss: 0.420258	valid_1's binary_logloss: 0.420625
[100]	valid_0's binary_logloss: 0.419775	valid_1's binary_logloss: 0.420173
[110]	valid_0's binary_logloss: 0.419383	valid_1's binary_logloss: 0.419809
[120]	valid_0's binary_logloss: 0.419012	valid_1's binary_logloss: 0.419471
[130]	valid_0's binary_logloss: 0.

[1090]	valid_0's binary_logloss: 0.410925	valid_1's binary_logloss: 0.415389
[1100]	valid_0's binary_logloss: 0.410875	valid_1's binary_logloss: 0.415382
[1110]	valid_0's binary_logloss: 0.410826	valid_1's binary_logloss: 0.415377
[1120]	valid_0's binary_logloss: 0.41078	valid_1's binary_logloss: 0.415373
[1130]	valid_0's binary_logloss: 0.410737	valid_1's binary_logloss: 0.415371
[1140]	valid_0's binary_logloss: 0.410693	valid_1's binary_logloss: 0.415371
[1150]	valid_0's binary_logloss: 0.410646	valid_1's binary_logloss: 0.415366
[1160]	valid_0's binary_logloss: 0.410599	valid_1's binary_logloss: 0.415361
[1170]	valid_0's binary_logloss: 0.410548	valid_1's binary_logloss: 0.415357
[1180]	valid_0's binary_logloss: 0.410498	valid_1's binary_logloss: 0.415354
[1190]	valid_0's binary_logloss: 0.410447	valid_1's binary_logloss: 0.415345
[1200]	valid_0's binary_logloss: 0.410401	valid_1's binary_logloss: 0.41534
[1210]	valid_0's binary_logloss: 0.410353	valid_1's binary_logloss: 0.415338
[

[2160]	valid_0's binary_logloss: 0.406243	valid_1's binary_logloss: 0.415105
[2170]	valid_0's binary_logloss: 0.406203	valid_1's binary_logloss: 0.415105
[2180]	valid_0's binary_logloss: 0.40616	valid_1's binary_logloss: 0.415105
[2190]	valid_0's binary_logloss: 0.40612	valid_1's binary_logloss: 0.415106
[2200]	valid_0's binary_logloss: 0.406077	valid_1's binary_logloss: 0.415101
[2210]	valid_0's binary_logloss: 0.406039	valid_1's binary_logloss: 0.415101
Early stopping, best iteration is:
[2116]	valid_0's binary_logloss: 0.406421	valid_1's binary_logloss: 0.4151
[0.41509953492609514]
test mean: 0.21312945162065725
Training until validation scores don't improve for 100 rounds.
[10]	valid_0's binary_logloss: 0.458461	valid_1's binary_logloss: 0.458298
[20]	valid_0's binary_logloss: 0.4405	valid_1's binary_logloss: 0.440306


KeyboardInterrupt: 

# process model

In [42]:
tt = pd.read_pickle(FILE.train_final.value)
te = pd.read_pickle(FILE.test_final.value)
tt.drop('time_hour',axis = 1,inplace=True)
te.drop('time_hour',axis = 1,inplace=True)
da = pd.concat([tt, te], axis=0, ignore_index=True,sort=False)
cv = CountVectorizer(min_df=20)
da['model'] = da.astype(str)
cv.fit(da['model'])

ValueError: After pruning, no terms remain. Try a lower min_df or a higher max_df.